# **Dealing with big datasets**

Si el conjunto de datos a cargar es demasiado grade para tenerlo en memoria podemos cargarlo por lotes o por trozos. Con **pandas** tenemos dos formas de hacerlo: 
* cargando el conjunto de datos en trozos del mismo tamaño
* pedir un iterador al conjunto de datos que permite pedir dinamicamanmete la longitud de cada trozo

In [81]:
import pandas as pd
iris_chunks = pd.read_csv('resources\datasets-uci-iris.csv', header=None,
                          names=['C1', 'C2', 'C3', 'C4', 'C5'],
                          chunksize=10)
for chunk in iris_chunks:
    print('Shape:', chunk.shape, end='  --  ')

Shape: (10, 5)  --  Shape: (10, 5)  --  Shape: (10, 5)  --  Shape: (10, 5)  --  Shape: (10, 5)  --  Shape: (10, 5)  --  Shape: (10, 5)  --  Shape: (10, 5)  --  Shape: (10, 5)  --  Shape: (10, 5)  --  Shape: (10, 5)  --  Shape: (10, 5)  --  Shape: (10, 5)  --  Shape: (10, 5)  --  Shape: (10, 5)  --  

In [83]:
iris_iterator = pd.read_csv('resources\datasets-uci-iris.csv', header=None,
                            names=['C1', 'C2', 'C3', 'C4', 'C5'],
                            iterator=True)
print(iris_iterator.get_chunk(10).shape)
print(iris_iterator.get_chunk(20).shape)
iris_iterator.get_chunk(2)


(10, 5)
(20, 5)


array([[4.8, 3.1, 1.6, 0.2, 'Iris-setosa'],
       [5.4, 3.4, 1.5, 0.4, 'Iris-setosa']], dtype=object)

Si la lectura de los datos desde CSV consume mucho tiempo y requiere de gran esfuerzo cada vez(establecer los tipos y los nombres de las variables correctas) podemos acelerar ese guardado y cargado usando la estructura de datos **HDF5** que es es un conjunto de tecnología único que hace posible la gestión de colecciones de datos extremadamente grandes y complejas. Se organiza como almacenamiento gerarquicos de datos que permite guardar matrices multidimensionales de un tipo o grupo homogéneo que son contenedores de matrices y otros grupos y se ajustan perfectamente a los DataFrame mediante la compresión automatica de los datos.

In [5]:
# instanciar el archivo
storage = pd.HDFStore('resources\example.h5')

# guardar en él como si fuera un diccionario
iris = pd.read_csv(
    'resources\datasets-uci-iris.csv',
    names=['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'target'])
storage['iris'] = iris
storage.close()


In [6]:
# para recuperar los archivos
storage = pd.HDFStore('resources\example.h5')

# verificar las llaves que hay
print(storage.keys())

fast_iris_upload = storage['iris']
type(fast_iris_upload)


['/iris']


pandas.core.frame.DataFrame

# **Use SQLite to munging data**

In [7]:
import sqlite3

# crear las consultas para eliminar y crear una tabla
drop_query = "DROP TABLE IF EXISTS temp_data;"
create_query = "CREATE TABLE temp_data \
(date INTEGER, city VARCHAR(80), \
temperature REAL, destination INTEGER);"

# conectar con DB o crearla si no existe
connection = sqlite3.connect("resources\example.db")
connection.execute(drop_query)
connection.execute(create_query)
connection.commit()


In [8]:
# insertar los datos en la DB
data = [(20140910, "Rome", 80.0, 0),
        (20140910, "Berlin", 50.0, 0),
        (20140910, "Wien", 32.0, 1),
        (20140911, "Paris", 65.0, 0)]
insert_query = "INSERT INTO temp_data VALUES(?, ?, ?, ?)"
connection.executemany(insert_query, data)
connection.commit()


In [9]:
selection_query = "SELECT date, city, temperature, destination FROM temp_data WHERE Date=20140910"
retrieved = pd.read_sql_query(selection_query, connection)
connection.close()
retrieved


,date,city,temperature,destination
0,20140910,Rome,80.0,0
1,20140910,Berlin,50.0,0
2,20140910,Wien,32.0,1


# **create a mask and apply for many functions**

In [10]:
# mask example
from doctest import Example


mask_target = iris['target'] == 'Iris-virginica'
iris.loc[mask_target, 'target'] = 'New label'

# Example of the Aplicate many statistics
funcs = {'sepal_length': ['mean', 'std'],
         'sepal_width': ['max', 'min'],
         'petal_length': ['mean', 'std'],
         'petal_width': ['max', 'min']}
iris.groupby(['target']).agg(funcs)


sepal_length           sepal_width      petal_length  \
                        mean       std         max  min         mean   
target                                                                 
Iris-setosa            5.006  0.352490         4.4  2.3        1.464   
Iris-versicolor        5.936  0.516171         3.4  2.0        4.260   
New label              6.588  0.635880         3.8  2.2        5.552   

                          petal_width       
                      std         max  min  
target                                      
Iris-setosa      0.173511         0.6  0.1  
Iris-versicolor  0.469911         1.8  1.0  
New label        0.551895         2.5  1.4

# **Aplicar  funciones e los datos en pandas**

In [11]:
'''operar elemento por elemnto con applymap()'''
# obtenr eltamaño de cadena de cada columna
iris.applymap(lambda x: len(str(x))).head(3)


,sepal_length,sepal_width,petal_length,petal_width,target
0,3,3,3,3,11
1,3,3,3,3,11
2,3,3,3,3,11


In [12]:
culumns_to_modif = ['sepal_length', 'sepal_width', 'petal_length']
iris[culumns_to_modif].apply(lambda x: x**2).head(3)


,sepal_length,sepal_width,petal_length
0,26.01,12.25,1.96
1,24.01,9.00,1.96
2,22.09,10.24,1.69


```Uno delos putos débiles es que estas transformaciones pueden demorar mucho tiempo pq la biblioteca no aprovecha las apacidades de multiprocesamiento de las CPU modernas. Por eso para grandes volumenes de datos debemos hacerlo nosotros mismo como se muestra en el script:  ``` **2__multiprocessing.py** ya que el portátil Jupyter no puede ejecutar multiprocesamiento.

# **Processing Text. Usage Bag(Bolsa) of Words**

In [13]:
# colección de aproximadamente 20 000 documentos de grupos de noticias, divididos (casi)
# uniformemente en 20 grupos de noticias diferentes
from sklearn.datasets import fetch_20newsgroups

categories = ['sci.med', 'sci.space']
twenty_sci_news = fetch_20newsgroups(categories=categories)
print(twenty_sci_news.data[0])


From: flb@flb.optiplan.fi ("F.Baube[tm]")
Subject: Vandalizing the sky
X-Added: Forwarded by Space Digest
Organization: [via International Space University]
Original-Sender: isu@VACATION.VENARI.CS.CMU.EDU
Distribution: sci
Lines: 12

From: "Phil G. Fraering" <pgf@srl03.cacs.usl.edu>
> 
> Finally: this isn't the Bronze Age, [..]
> please try to remember that there are more human activities than
> those practiced by the Warrior Caste, the Farming Caste, and the
> Priesthood.

Right, the Profiting Caste is blessed by God, and may 
 freely blare its presence in the evening twilight ..

-- 
* Fred Baube (tm)



In [14]:
'''transformar cada documento en un conjunto de palabras(caracteristicas)'''
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
word_count = count_vect.fit_transform(twenty_sci_news.data)
print('matrix is sparse de dimensiones : ', word_count.shape)

# ver el primer documento ( solo las 10 primeras palabras)
word_list = count_vect.get_feature_names()
for n in word_count[0].indices[:10]:
    print('Word "%s" appears %i times' % (word_list[n], word_count[0, n]), end='         ')


matrix is sparse de dimensiones :  (1187, 25638)
Word "from" appears 2 times         Word "flb" appears 2 times         Word "optiplan" appears 1 times         Word "fi" appears 1 times         Word "baube" appears 2 times         Word "tm" appears 2 times         Word "subject" appears 1 times         Word "vandalizing" appears 1 times         Word "the" appears 7 times         Word "sky" appears 1 times         

c:\Users\Manue!_PC\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [15]:
'''calcular las frecuenciuas de cada palabra'''
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np


tf_vect = TfidfVectorizer(use_idf=False, norm='l1')
word_freq = tf_vect.fit_transform(twenty_sci_news.data)
word_list = tf_vect.get_feature_names()
for n in word_freq[0].indices[:10]:
    print('Word "%s" has frequency %0.3f' % (word_list[n], word_freq[0, n]), end='         ')

word_freq[0].sum()


Word "from" has frequency 0.022         Word "flb" has frequency 0.022         Word "optiplan" has frequency 0.011         Word "fi" has frequency 0.011         Word "baube" has frequency 0.022         Word "tm" has frequency 0.022         Word "subject" has frequency 0.011         Word "vandalizing" has frequency 0.011         Word "the" has frequency 0.077         Word "sky" has frequency 0.011         

0.9999999999999987

Conmo vemos si la norma es 'L1' la suma de las frecuencias va a ser igual a uno. si queremos aumentar la diferencia entre palabras raras y las comunes podemos usar 'l2' entonces la suma de los cuadrados de las frecuencias obtenidas va a ser igual a uno. lo podemos comprobar  con 
```python
np.power(word_freq[0].todense(),2).sum()


In [16]:
'''uso del parametro 'use_idf'  (significa término-frecuencia multiplicado por la inversa de la frecuencia del documento)  que permite destacar palabras q describen eficazmente cada documento'''

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer()  # Default: use_idf=True

word_tfidf = tfidf_vect.fit_transform(twenty_sci_news.data)
word_list = tfidf_vect.get_feature_names()
for n in word_tfidf[0].indices[:10]:
    print('Word "%s" has tf-idf %0.3f' % (word_list[n], word_tfidf[0, n]), end='         ')


Word "fred" has tf-idf 0.089         Word "twilight" has tf-idf 0.139         Word "evening" has tf-idf 0.113         Word "in" has tf-idf 0.024         Word "presence" has tf-idf 0.119         Word "its" has tf-idf 0.061         Word "blare" has tf-idf 0.150         Word "freely" has tf-idf 0.119         Word "may" has tf-idf 0.054         Word "god" has tf-idf 0.119         

Asi d esta manera las palabras con frecuencias altas en el documento son raras en el resto de los documentos

In [17]:
'''usando n-grams donde la presencia o ausencia de una palabra asi como de sus vecinas importa'''

documents = np.array(['we love data science', 'data science is hard'])

# uni- and bi-gram count vectorizer for only use bi-gram with (2,2)
count_vect_1_grams = CountVectorizer(ngram_range=(1, 2))
word_count = count_vect_1_grams.fit_transform(documents)
word_list = count_vect_1_grams.get_feature_names()
print("Word list = ", word_list)
print(
    "text_1 is described with",
    [word_list[n] + "(" + str(word_count[0, n]) + ")" for n in word_count[0].indices])


Word list =  ['data', 'data science', 'hard', 'is', 'is hard', 'love', 'love data', 'science', 'science is', 'we', 'we love']
text_1 is described with ['we(1)', 'love(1)', 'data(1)', 'science(1)', 'we love(1)', 'love data(1)', 'data science(1)']


El número de características se dispara exponencialmente cuando usamos **n-grams**. si se tienen demasiadas caracteristicas es habitual usar el truco del Hashing en el que se hace un **bucket of words** y sus hash colisionan. Los **bucket** son conjuntos de palabras no relacionadas semanticamente pero con hashes que colisionan

In [18]:
from sklearn.feature_extraction.text import HashingVectorizer
hash_vect = HashingVectorizer(n_features=8, norm='l1')
word_hashed = hash_vect.fit_transform(documents)
word_hashed.todense()


matrix([[ 0.  , -0.5 ,  0.  ,  0.25,  0.  ,  0.  ,  0.  ,  0.25],
        [ 0.  , -0.5 ,  0.  ,  0.5 ,  0.  ,  0.  ,  0.  ,  0.  ]])

Ten en cuenta que no puedes invertir el proceso de hashing (ya que es una operación de digestión).Por lo tanto, después de esta transformación, usted tendrá que trabajar en el hashed tal y como están. 

El hashing presenta bastantes ventajas: permite transformar rápidamente una bolsa de palabras en vectores de características (los bucket de hash son nuestras características, en este caso) y evitar el sobreajuste por de palabras no relacionadas entre sí en la misma característica. Ademas es de memoria muy baja escalable a grandes conjuntos de datos ya que no hay necesidad de almacenar un diccionario de vocabulario en la memoria.

También hay un par de contras (en comparación con el uso de un CountVectorizer con un vocabulario en memoria): no hay forma de calcular la transformada inversa (desde índices de características hasta nombres de características de cadena), lo que puede ser un problema cuando se intenta introspeccionar qué características son más importantes para un modelo. Puede haber colisiones: distintos tokens se pueden asignar al mismo índice de características. Sin embargo, en la práctica, esto rara vez es un problema si n_features es lo suficientemente grande (por ejemplo, 2 ** 18 para problemas de clasificación de texto) y ademas no tiene ponderación IDF



# **Scraping(raspado) the web with BeautifulSoup**
Es un tema muy extenso al que se le pudiera dedicar practicamente un libro, por eso para aplicarlo hay q buscar mas informacion 

In [31]:
import urllib.request

# 1. Let's download the code behind the William Shakespeare page on Wikipedia:

url = 'https://en.wikipedia.org/wiki/William_Shakespeare'
request = urllib.request.Request(url)
response = urllib.request.urlopen(request)

In [33]:
from bs4 import BeautifulSoup

# 2. leer el y parsear el URL con analizaforHTML

soup = BeautifulSoup(response, 'html.parser')
soup.title


<title>William Shakespeare - Wikipedia</title>

Primero debemos analizar manualmente la propia página HTML para averiguar cuál es la mejor etiqueta HTML que contiene la
información que buscamos. Tras un **análisis manual**, descubrimos que las categorías están dentro de un div llamado
mw-normal-catlinks'; excluyendo el primer enlace, todos los demás están bien. Ahora, es
es hora de programar.

In [41]:
section = soup.find_all(id='mw-normal-catlinks')[0]
for catlink in section.find_all("a")[1:]:
    print(catlink.get("title"), "->", catlink.get("href"))


Category:William Shakespeare -> /wiki/Category:William_Shakespeare
Category:1564 births -> /wiki/Category:1564_births
Category:1616 deaths -> /wiki/Category:1616_deaths
Category:16th-century English male actors -> /wiki/Category:16th-century_English_male_actors
Category:English male stage actors -> /wiki/Category:English_male_stage_actors
Category:16th-century English dramatists and playwrights -> /wiki/Category:16th-century_English_dramatists_and_playwrights
Category:17th-century English dramatists and playwrights -> /wiki/Category:17th-century_English_dramatists_and_playwrights
Category:16th-century English poets -> /wiki/Category:16th-century_English_poets
Category:Burials in Warwickshire -> /wiki/Category:Burials_in_Warwickshire
Category:17th-century English poets -> /wiki/Category:17th-century_English_poets
Category:17th-century English male writers -> /wiki/Category:17th-century_English_male_writers
Category:English Renaissance dramatists -> /wiki/Category:English_Renaissance_dra

# **Data processing with NumPy**

##  **Create ndArray**

In [127]:
import numpy as np
list_of_ints = [1, 2, 3]
Array_1 = np.array(list_of_ints, dtype=np.int8)

print(type(Array_1))
Array_1.nbytes


<class 'numpy.ndarray'>


3

Como los ndArray tienen tamaño fijo es muy importante que el tipo de datos sea el adecuado para nuestros datos ya que nos ayudará a ahorrar memoria por ejemplo : 
* int 4B Default integer type (normally int32or int64)
* int8          ----------->1Bytes------------   Byte (-128 to 127)
* int16         ----------->2Bytes------------   Integer (-32768 to 32767)
* int32         ----------->4Bytes------------   Integer (-2** 31 to 2** 31-1)
* int64         ----------->8Bytes------------   Integer (-2** 63 to 2** 63-1)
* uint8         ----------->1Bytes------------   Unsigned integer (0 to 255)
* uint16        ----------->2Bytes------------   Unsigned integer (0 to 65535)
* uint32        ----------->4Bytes------------   Unsigned integer (0 to 2**32-1)
* uint64        ----------->8Bytes------------   Unsigned integer (0 to 2**64-1)
* float16       ----------->2Bytes------------   Half-precision float (exponent 5 bits, mantissa 10 bits)
* float_        ----------->8Bytes------------   Shorthand for float64
* float32       ----------->4Bytes------------   Single-precision float (exponent 8 bits, mantissa 23 bits)
* float64       ----------->8Bytes------------   Double-precision float (exponent 11 bits,mantissa 52 bits)


Para cambiar el tipo de de un ndArray se utiliza    
```python
Array_1.astype('float32')
float types pevalece sobre tipos int,y strings (<U32means a Unicode string of size 32 or less)predominan a todos los demas tipos

In [130]:
# Check if a NumPy array is of the desired numeric type
print(isinstance(Array_1[0], np.number))

# crear un ndArray a partir de un diccionario
np.array({1: 2, 3: 4, 5: 6}.items())

# Para crearlo mediante un DataFrame se utiliza el atributo df.values 
# Se recomienda antes de hacer esto verificar los tipos en cada columna y si no
# son homogeneos realizar alguna trasnformacion 

True


array(dict_items([(1, 2), (3, 4), (5, 6)]), dtype=object)

## **Resizing arrays**

In [22]:
import numpy as np

# Restructuring a NumPy array shape
original_array = np.array([1, 2, 3, 4, 5, 6, 7, 8])

Array_a = original_array.reshape(4, 2)
Array_b = original_array.reshape(4, 2).copy()
Array_c = original_array.reshape(2, 2, 2)

# Attention because reshape creates just views, not copies
original_array[0] = -1

print('array  modificado pq es una vista del original\n', Array_a)
print('array sin modificar pq es una copia\n', Array_b)


array  modificado pq es una vista del original
 [[-1  2]
 [ 3  4]
 [ 5  6]
 [ 7  8]]
array sin modificar pq es una copia
 [[1 2]
 [3 4]
 [5 6]
 [7 8]]


In [19]:
# para cambiar las diemnsiones del array original se utiliza
original_array.resize(4, 2)
original_array


array([[-1,  2],
       [ 3,  4],
       [ 5,  6],
       [ 7,  8]])

In [21]:
# resize es equivalente a
original_array.shape = (1, 8)
original_array.shape = (1, -1)  # ya que -1 re refiere al ultimo elemento

original_array


array([[-1,  2,  3,  4,  5,  6,  7,  8]])

## **Arrays derived from NumPy functions**

In [68]:
# uso de arange e invirtiendo el arreglo con el operador de slice
np.arange(9)[::-1]

# Enteros aleatrorios y posiblemente repetidos
np.random.randint(low=1, high=10, size=(2, 4))

np.zeros((3, 3))     # matriz nula

np.ones((3, 3))      # matriz identidad

np.eye(3)           # matriz diagonal

# matriz fraccionaria del tamaño especificado y en el intervalo cerrado (0,2)
fraction = np.linspace(start=0, stop=2, num=5)

# matriz fraccionaria del tamaño  especificado y en el intervalo cerrado (base**start, base**stop)
np.logspace(start=0, stop=2, num=5, base=10.0)

# matriz de valores aleatorios que siguen la distribucion normal (mean=0, std=1)
gaussian = np.random.normal(loc=0, scale=1, size=(2, 4))

# matriz de valores aleatorios que siguen la distribucion uniforme (low=0.0, high=1.0)
np.random.uniform(low=0.0, high=1.0, size=(2, 4))


array([[0.2812499 , 0.05111399, 0.97705041, 0.27080453],
       [0.09449049, 0.50801466, 0.29855802, 0.11840719]])

## **Getting an array directly from a file**

Si ejecutaramos este comando:
```python 
housing = np.loadtxt("resources/datasets-uci-iris.csv", delimiter=',', dtype=float)
``` 
nos daria una error similar a este :*ValueError: could not convert string to float: Iris-setosa*. una solucion seria si sabemos en que columna esta el problema entonces aplicarle una conversion mediante una funcion como sigue:

In [79]:
def from_txt_to_iris_class(x):
    if x == b'Iris-setosa':
        return 0
    elif x == b'Iris-versicolor':
        return 1
    elif x == b'Iris-virginica':
        return 2
    else:
        return np.nan


np.loadtxt("resources/datasets-uci-iris.csv", delimiter=',',
           converters={4: from_txt_to_iris_class})[:5,:]


array([[5.1, 3.5, 1.4, 0.2, 0. ],
       [4.9, 3. , 1.4, 0.2, 0. ],
       [4.7, 3.2, 1.3, 0.2, 0. ],
       [4.6, 3.1, 1.5, 0.2, 0. ],
       [5. , 3.6, 1.4, 0.2, 0. ]])

## **Aplicar una funcion a aun ndArray**

In [84]:
def cube_power_square_root(x):
    return np.sqrt(np.power(x, 3))
    
np.apply_along_axis(cube_power_square_root, 
axis=0, arr=np.arange(5))

array([0.        , 1.        , 2.82842712, 5.19615242, 8.        ])

## **Matrix operations**


In [99]:
from re import S
from wsgiref import simple_server


a = np.array((1, 2, 3))
b = np.array((2, 3, 4))

# formar una matriz concatenado cada arreglo como si fueran una columna
np.column_stack((a, b))

# matriz bidimensional de 0 a 3
M = np.arange(2*2, dtype=float).reshape(2, 2)

coefs = np.array([1., 0.5])

# producto de dos matrices
np.dot(M, coefs)


array([0.5, 3.5])

In [104]:
# crear indices booleanos para saber que filas y columnas queremos 
M = np.arange(5*5, dtype=float).reshape(5, 5)

row_index = (M[:,0]>=5) & (M[:,0]<=15)  # solo las filas 2,3,4
col_index = M[0,:]>=3                   # solo las dos ultimas columnas 

# notar como no podemos aplicar los indicies boolesnos dentro del mismo [], por eso se 
# filtra primero las filas y luego las columnas con 2 corchetes
print(M[row_index,:][:,col_index] )

mask = (M>=15) & (M<=20) & ((M / 10.) % 1 >= 0.5)
M[mask]

[[ 8.  9.]
 [13. 14.]
 [18. 19.]]


array([15., 16., 17., 18., 19.])

## **Fancy indexing(indexacion de lujo)**

In [105]:
# Secuancia de indices que pueden estar desordenados e incluso repetidos para obtener
# los elementos  ( 1,0),(1,2),(2,4)
row_index = [1,1,2]
col_index = [0,2,4]
M[row_index,col_index]

array([ 5.,  7., 14.])

In [107]:
# tambien sirve para seleccioanr filas ycolumnas que deseemos aplicamos primero la indexacion
# por fila y luego por columna
M[row_index, :][:, col_index]


array([[ 5.,  7.,  9.],
       [ 5.,  7.,  9.],
       [10., 12., 14.]])

*Recuerde que la segmentación y la indexación son solo vistas de los datos y cualquier cambio en el **ndArray original** se mostrará en ellos. Sino desea esto tiene que hacer una copia con el metodo **copy***

## **Stacking NumPy arrays(Añadir filas y columnas)**

In [138]:
# nuestro conjunto de datos
dataset = np.arange(5*5).reshape(5, 5)

single_line = np.arange(1*5).reshape(1, 5)
a_few_lines = np.arange(3*5).reshape(3, 5)

# añadir una sola fila
np.vstack((dataset, single_line))

# añadir varias filas
np.vstack((dataset,a_few_lines))

#añadiendo todas las filas que querramos al parametro que es una tupla donde el 
# primer elemento es el arreglo al que se le van a añadir las filas
np.vstack((dataset,single_line,single_line))

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24],
       [ 0,  1,  2,  3,  4],
       [ 0,  1,  2,  3,  4]])

In [140]:
bias = np.ones(5).reshape(5,1)

#añadir una nueva columna similar a las filas
np.hstack((dataset,bias))

# tambien podemos hacerlo sin remodelar la columna y unirla a nuestro ndArray
bias = np.ones(5)
np.column_stack((dataset,bias))

array([[ 0.,  1.,  2.,  3.,  4.,  1.],
       [ 5.,  6.,  7.,  8.,  9.,  1.],
       [10., 11., 12., 13., 14.,  1.],
       [15., 16., 17., 18., 19.,  1.],
       [20., 21., 22., 23., 24.,  1.]])

In [141]:
# inserecion en una posición específica
np.insert(dataset, 3, bias, axis=1)

array([[ 0,  1,  2,  1,  3,  4],
       [ 5,  6,  7,  1,  8,  9],
       [10, 11, 12,  1, 13, 14],
       [15, 16, 17,  1, 18, 19],
       [20, 21, 22,  1, 23, 24]])

## **Sparse arrays(matrices dispersas)**